# WSRT - Safari unit for Apercal

In [1]:
#!/usr/bin/env python
import sys,string,os,math
import time
import shutil
import pyfits
from scipy.interpolate import interp1d
from mirpy import miriad
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib import gridspec

import numpy as np

tempoinit=time.time()
print 'LOADED'

LOADED


In [2]:
#set input / output filenames
#get set path where I put the data, the sources list and the observation I want to reduce 
#path_or='/Users/maccagni/Documents/Wdata/'
#path_fig='/Users/maccagni/Documents/Wdata/FIG/'

path_or='/Users/maccagni/Sharp_data/PKS/'


#calibrator and source should be called this way.

cal_1='cal/'
src_1='src/'

#this is the folder for each epoch of observation containing three folders, 
#one for each day (which I call epoch_1, epoch_2, epoch_3 )
obs='v2/'

#set fundamental constants
C=2.99792458e5 #km/s
HI=1.420405751e9 #Hz

In [3]:
#define important variables

#channels selected for the continuum image
CHANS_con='channel,1849,100,1,1'
#channels selected for the spectral line cube 
CHANS_cube='channel,312,719,1,1'

#magic parameters for the autoflag routine
flags1='6,8,5,3,5,3'
flags2='8,10,5,3,5,3'
flags3='12,14,5,3,5,3'
flags4='14,16,5,8,5,8'


In [4]:
def calflag(flags):
   
    miriad.mfcal(vis=cal_1,stokes='xx,yy',line=CHANS_con,interval='15')
    miriad.pgflag(vis=cal_1,stokes='qq',options='nodisp',flagpar=flags,command=' \'<\'')
    miriad.mfcal(vis=cal_1,stokes='xx,yy',line=CHANS_con,interval='15')
     
    return 0
    
    
def srcflag(flags):
   
    miriad.pgflag(vis=src_1,stokes='qq',options='nodisp',flagpar=flags,command=' \'<\'')

    return 0    




def continuum_image(source):    
    #CREATE the IMAGE
    
    print '***********CONTINUUM: Invert**********'
    #set parameter filenames
    cont_im='c_m'
    cont_bm='c_bm'
    
    cleanup(cont_im)
    cleanup(cont_bm)
    
    #set parameters
    IMSIZE='1024'
    CELL='4'
    ROBUST='0.5'
    ANT='-ant(6)'
    STOKES='ii'
    OPTIONS='double,mfs,sdb'
    SLOP='1'
    
    miriad.invert(vis=source,map=cont_im,beam=cont_bm,imsize=IMSIZE,cell=CELL,robust=ROBUST,select=ANT,stokes=STOKES,options=OPTIONS,slop=SLOP)

    
    #CLEAN the image
    print '***********CONTINUUM: Clean**********'
    
    
    #set parameter filenames
    clean_cont='cl'
    
    cleanup(clean_cont)
    
    #set parameters
    CUTOFF='5e-2'
    miriad.clean(map=cont_im,beam=cont_bm,cutoff=CUTOFF,out=clean_cont)
    
    #RESTOR the image
    print '***********CONTINUUM: Restore**********'
       
    #set parameter filenames    
    cont_restor='c_r'
    
    cleanup(cont_restor)
    
    miriad.restor(model=clean_cont,map=cont_im,beam=cont_bm,out=cont_restor)    
    
    
    return 0

def make_cube(calib,source):
    
    
    print '***********cut the band**********'
    miriad.mfcal(vis=calib,stokes='xx,yy',line=CHANS_cube,interval='15') 
    miriad.gpcopy(vis=calib,out=source,mode='copy')
    
        
    #UVLIN: continuum subtraction
    
    MODEL='cl'
    source_sub='src_cube_sub'
    
    cleanup(source_sub)
    
    miriad.uvmodel(vis=source,model=MODEL,options='subtract',out=source_sub)
    
    
    #set filenames
    src_lin='src_line'
    cleanup(src_lin)
    
    #set parameters

    ORDER='3'
    MODE='line'
    CHANNEL='channel,300,719,1,1'
    CHANS='1,150,200,300'

    print '***********continuum subtraction**********'
    miriad.uvlin(vis=source,line=CHANNEL,chans=CHANS,order=ORDER,mode=MODE,out=src_lin)
    
    
    RESTFREQ=src_lin+'/restfreq'
    FREQ='1.420405752'
    TYPE='double'
    
    print '***********VEL: puthd**********'
    
    miriad.puthd(IN=RESTFREQ,value=FREQ,type=TYPE)    
    
    #Invert: make the right cube
    #set filenames
    cube_im='m'
    cube_bm='b'    
    
    cleanup(cube_im)
    cleanup(cube_bm)
    
    
    #set parameters
    IMSIZE='1024'
    CELL='2'
    #CHANNEL='channel,250,1,3,3'
    ROBUST='0'
    STOKES='ii'
    OPTIONS='double'
    SLOP='0.8'
    FWHM='5'  
    
    #centrato sulla riga
    CHANNEL='channel,300,1,1,1'

    print '***********invert**********'    
    
    miriad.invert(vis=src_lin,map=cube_im,beam=cube_bm,imsize=IMSIZE,
                  cell=CELL,robust=ROBUST,stokes=STOKES,options=OPTIONS,slop=SLOP,line=CHANNEL,fwhm=FWHM)
    
    miriad.velsw(IN=cube_im,axis='vopt')    
    
    
def cleanup(infile):
        
    
    if os.path.exists(infile) == True:
        shutil.rmtree(infile)
        
    
    


In [9]:
def SpecExtr(infile,outfile,v_sys,px,py):
 
    hdulist = pyfits.open(infile) #read input
    
    flux=[]
    flux_box_vec=[]
    madfm=[]
    madfm_box=[]

    #read data and header
    scidata = hdulist[0].data
    #reduce dimensions of the input cube
    sci=np.squeeze(scidata)
    prihdr=hdulist[0].header
        
    len_vel=hdulist[0].header['NAXIS3']
    len_pix=hdulist[0].header['NAXIS2']
    #print len_vel
    #x-axis formula
    freq = (np.linspace(1,sci.shape[0],sci.shape[0]) - prihdr['CRPIX3'])* prihdr['CDELT3'] + prihdr['CRVAL3']
    freq = freq/1e3
    
    #v=C*((HI-freq)/freq)

    v=HI/(freq/C+1)
    
    v_wr_sys=(freq-v_sys)
    
    #I determine the pixels where to extract the spectra
    
    #I make the header bi-dimensional so wcs does not get confused
    del prihdr['CTYPE4']
    del prihdr['CDELT4']    
    del prihdr['CRVAL4']
    del prihdr['CRPIX4']
    del prihdr['CTYPE3']
    del prihdr['CDELT3']
    del prihdr['CRVAL3']
    del prihdr['CRPIX3'] 
    
    del prihdr['NAXIS3']
    del prihdr['NAXIS4']
    del prihdr['NAXIS']
    prihdr['NAXIS']=2
    
    #ra=-float(prihdr['CRVAL1'])
    #dec=float(prihdr['CRVAL2'])
    
    #I load the WCS coordinate system:
    #w=wcs.WCS(prihdr)    
    #the third digit gives the origin pixel (either 0 or 1, check with James)
    #px,py=w.wcs_world2pix(ra,dec,0)
    #print px,py
    #extract the spectrum
    
    for i in xrange (0,len_vel):
        pix_x=int(py)
        pix_y=int(px)        
        flux.append(sci[i,pix_x,pix_y])
        #flux_box=np.sum(sci[i,pix_x-3:pix_x+3,pix_y-5:pix_y+5]) # (7X11) box
        #flux_box_vec.append(flux_box)        
        
        #determine the noise as in Whiting 2012 et al. 
        #MADMF: median absolute deviation from the median
        #extract a region were to determine the noise: A BOX 
        #!!!!!!! this box includes also the pixel where we see the line: not so smart !!!!!
        if pix_x != 0 or pix_y !=0:
            rms=np.median(sci[i,pix_x-5:pix_x+5,pix_y-5:pix_y+5])
            med2=abs(sci[i,pix_x,pix_y]-rms)
            madfm.append(np.median(med2)/0.6744888)
        else:
            madfm=np.zeros(sci.shape[0])
        #BOX
        #rms=np.median(sci[i,pix_x-3:pix_x+3,pix_y-5:pix_y+5])
        #med2=np.abs(sci[i,pix_x-3:pix_x+3,pix_y-5:pix_y+5]-rms)
        #madfm_box.append(np.median(med2)/0.6744888)
    
    mean_noise=np.mean(madfm)
    
    #mean_noise_box=np.mean(madfm_box)
    
    
    
    #noise along the spectrum
    noise_lin=(np.std(sci[0:70,pix_x,pix_y])+np.std(sci[151:219,pix_x,pix_y]))/2.
    
    tot=np.column_stack((v,freq,v_wr_sys,flux,madfm))
    #del madfm[:]
    #del med2
    
    #write the spectrum on file
    f=open(outfile,'w')
    f.write('freq[Hz],vel[km/s],vel_rel[km/s],flux[Jy/beam],noise\n')
    np.savetxt(f,tot,delimiter=",",fmt="%s")
    f.close()


    return noise_lin,mean_noise


def plotta(x,y,y2,outfile,flag,flag2,hi_sys,rms_mean):
    

    fig_b = plt.figure(figsize=(12, 8), dpi=100)
    

    
    
    params = {'legend.fontsize': 12,
           'axes.linewidth':2,
            'axes.labelsize':22,
           'lines.linewidth':1,
           'legend.linewidth': 3,
           'xtick.labelsize':22,
           'ytick.labelsize':22,
           'xlabel.fontsize':22,
           'ylabel.fontsize':22,
           'text.usetex': True,
           'text.latex.unicode' : True }
    rc('font',**{'family':'serif','serif':['serif']})
    plt.rcParams.update(params)
    #plt.figtext(0.16,0.8,nome, fontsize=28)
    #plt.figtext(0.6,0.2,r'$v_{sys}$ [MHz]='+str(v),fontsize=14)

    
    
    gs_all_line = gridspec.GridSpec(1, 1)
    
    gs_all_line.update(left=0.1, right=0.9, wspace=0.0,hspace=0.0)

    gs_spec = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs_all_line[0], wspace=0.0, hspace=0.0)   
    
    ax_spec = fig_b.add_subplot(gs_spec[0:3, 0])
    ax_noise = fig_b.add_subplot(gs_spec[3, 0]) 
    
    
    ax_spec.plot(x, y, marker='o',color='black',lw=3)
    ax_noise.plot(x, y2, marker=' ',color='black',lw=3)
    
    
 
    xx_spec=[min(x),max(x)]
    yy_spec=[min(y)-max(y)/2.,max(y)+max(y)/2.]

    yy_noise=[min(y2)-max(y2)/4.,max(y2)+max(y2)/4.]

    
    a_spec=[0.0,0.0]
    a_noise=[rms_mean,rms_mean]

    #aa=(HI/vsys-1)*C
    
    
    ax_spec.set_xlim([xx_spec[0],xx_spec[1]])
    ax_noise.set_xlim([xx_spec[0],xx_spec[1]])



    

    
    ax_spec.plot(xx_spec, a_spec,linestyle='--',color='black',lw=3)
    ax_noise.plot(xx_spec, a_noise,linestyle='-.',color='black',lw=3)

    #ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
    #ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3)
    ax_spec.set_xticklabels([ ])

   
    if flag==1:
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)

    if flag==0:
        b_spec=[hi_sys,hi_sys]

        ax_spec.plot(b_spec,yy_spec,linestyle='--',color='black',lw=3)
        ax_noise.plot(b_spec,yy_noise,linestyle='--',color='black',lw=3)

        ax_spec.set_ylim([-0.018,0.01])
    
        ax_noise.set_ylim([-0.01,0.01])      
        ax_spec.set_xlim([3900,4600])
        ax_noise.set_xlim([3900,4600])
        
        ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
        ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3) 
        ax_noise.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        
        #if flag2==0:
        #    rect1 = patches.Rectangle((xx_spec[0],1.137),xx_spec[1]-xx_spec[0], 0.985, color='grey')
        #    ax_noise.add_patch(rect1)
        #if flag2==1:
        #    rect1 = patches.Rectangle((xx_spec[0],0.882),xx_spec[1]-xx_spec[0], 0.792, color='grey')
        #    ax_noise.add_patch(rect1)
       
    if flag==2:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        b_spec=[hi_sys,hi_sys]

        ax_spec.plot(b_spec,yy_spec,linestyle='--',color='black',lw=3)
    
        ax_spec.set_xlabel(r'Frequency [$10^{9}$ \,\,{\rm Hz}]', fontsize=20)
    if flag==3:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)

        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)
        ax_noise.set_ylabel(r'Noise [$\tau$]',fontsize=20) 

    #plt.show()
    
    fig_b.savefig(outfile,format='pdf',bbox_inches='tight')
    
    return 0



def plotta_over(x1,y1,x2,y2,x3,y3,x_or,y_or,resid_x,resid,outfile,flag,flag2,hi_sys,rms_mean):
    

    fig_b = plt.figure(figsize=(12, 8), dpi=100)
    

    
    
    params = {'legend.fontsize': 12,
           'axes.linewidth':2,
            'axes.labelsize':22,
           'lines.linewidth':1,
           'legend.linewidth': 3,
           'xtick.labelsize':22,
           'ytick.labelsize':22,
           'xlabel.fontsize':22,
           'ylabel.fontsize':22,
           'text.usetex': True,
           'text.latex.unicode' : True }
    rc('font',**{'family':'serif','serif':['serif']})
    plt.rcParams.update(params)
    #plt.figtext(0.16,0.8,nome, fontsize=28)
    #plt.figtext(0.6,0.2,r'$v_{sys}$ [MHz]='+str(v),fontsize=14)

    
    
    gs_all_line = gridspec.GridSpec(1, 1)
    
    gs_all_line.update(left=0.1, right=0.9, wspace=0.0,hspace=0.0)

    gs_spec = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs_all_line[0], wspace=0.0, hspace=0.0)   
    
    ax_spec = fig_b.add_subplot(gs_spec[0:3, 0])
    ax_noise = fig_b.add_subplot(gs_spec[3, 0]) 
    
    ax_spec.plot(x_or, y_or, marker='o',color='red',lw=3,label='2013')
 
    ax_spec.plot(x1, y1, marker='o',color='black',lw=3,label='2015 1')
    ax_spec.plot(x2, y2, marker='o',color='blue',lw=3,label='2015 2')
    ax_spec.plot(x3, y3, marker='o',color='green',lw=3,label='2015 3')

    
    ax_spec.legend()
    
    ax_noise.plot(resid_x, resid, marker=' ',color='black',lw=3)
    ax_noise.plot(x2, y2, marker=' ',color='green',lw=3)
    #ax_noise.plot(x3, yn3, marker=' ',color='red',lw=3)
    
    
 
    xx_spec=[min(x1),max(x1)]
    yy_spec=[min(y2)-max(y2)/2.,max(y2)+max(y2)/2.]

    yy_noise=[min(y2)-max(y2)/4.,max(y2)+max(y2)/4.]

    
    a_spec=[0.0,0.0]
    a_noise=[0.0,0.0]

    #aa=(HI/vsys-1)*C
    
    
    ax_spec.set_xlim([xx_spec[0],xx_spec[1]])
    ax_noise.set_xlim([xx_spec[0],xx_spec[1]])



    

    
    ax_spec.plot(xx_spec, a_spec,linestyle='--',color='black',lw=3)
    ax_noise.plot(xx_spec, a_noise,linestyle='-.',color='black',lw=3)

    #ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
    #ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3)
    ax_spec.set_xticklabels([ ])

   
    if flag==1:
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)

    if flag==0:
        b_spec=[hi_sys,hi_sys]

        #ax_spec.plot(b_spec,yy_spec,linestyle='--',color='black',lw=3)
        #ax_noise.plot(b_spec,yy_noise,linestyle='--',color='black',lw=3)

        ax_spec.set_ylim([-0.02,0.02])
    
        ax_noise.set_ylim([-0.01,0.01])      
        ax_spec.set_xlim([3900,4600])
        ax_noise.set_xlim([3900,4600])
        
        ax_spec.set_ylabel(r'Flux [mJy]',fontsize=20,labelpad=-4)
        ax_noise.set_ylabel(r'Noise [mJy]',fontsize=20,labelpad=+3) 
        ax_noise.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)
        
        #if flag2==0:
        #    rect1 = patches.Rectangle((xx_spec[0],1.137),xx_spec[1]-xx_spec[0], 0.985, color='grey')
        #    ax_noise.add_patch(rect1)
        #if flag2==1:
        #    rect1 = patches.Rectangle((xx_spec[0],0.882),xx_spec[1]-xx_spec[0], 0.792, color='grey')
        #    ax_noise.add_patch(rect1)
       
    if flag==2:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        b_spec=[hi_sys,hi_sys]

        ax_spec.plot(b_sepc,yy_spec,linestyle='--',color='black',lw=3)
    
        ax_spec.set_xlabel(r'Frequency [$10^{9}$ \,\,{\rm Hz}]', fontsize=20)
    if flag==3:    
        ax_spec.set_ylabel(r'$\tau$', fontsize=20)
        ax_spec.set_xlabel(r'Velocity [km s^{-1}]', fontsize=20)

        ax_spec.plot(a_spec,yy_spec,linestyle='--',color='black',lw=3)
        ax_noise.set_ylabel(r'Noise [$\tau$]',fontsize=20) 

    #plt.show()
    
    fig_b.savefig(outfile,format='pdf',bbox_inches='tight')
    
    return 0

In [208]:
#CALIBRATE and flag
for i in xrange(1,4):
    path_or='/Users/maccagni/Sharp_data/PKS/'+obs+'epoch_'+str(i)+'/'
    os.chdir(path_or)

    #miriad.uvflag(vis=cal_1,flagval='flag',select='polarization(xy)')
    #miriad.uvflag(vis=cal_1,flagval='flag',select='polarization(yx)')

    miriad.mfcal(vis=cal_1,stokes='xx,yy',line=CHANS_con,interval='15')


    #flag some annoying channels
    chan_special='chan,5,617,1,1'
    miriad.uvflag(vis=cal_1,flagval='flag',line=chan_special)


    print '***********START AUTOFLAGGING CAL**********'

    calflag(flags1)
    print '.....1st cycle.....'

    calflag(flags2)
    print '.....2nd cycle.....'
    calflag(flags3)
    print '.....3rd cycle.....'
    calflag(flags4)
    print '.....4th cycle.....'

    print '***********END AUTOFLAGGING CAL**********'



***********START AUTOFLAGGING CAL**********
.....1st cycle.....

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 2599884
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 2606592
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)



.....2nd cycle.....
.....3rd cycle.....

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 2608022
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 2608688
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)



.....4th cycle.....
***********END AUTOFLAGGING CAL**********
***********START AUTOFLAGGING CAL**********

...


.....4th cycle.....
***********END AUTOFLAGGING CAL**********
***********START AUTOFLAGGING CAL**********
.....1st cycle.....

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 920306
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 939512
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)



.....2nd cycle.....
.....3rd cycle.....

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 942332
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 943200
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)



.....4th cycle.....
***********END AUTOFLAGGING CAL**********


In [209]:
# COPY CALIBRATION AND FLAG

for i in xrange(1,4):
    path_or='/Users/maccagni/Sharp_data/PKS/'+obs+'epoch_'+str(i)+'/'
    os.chdir(path_or)

    print '***********START AUTOFLAGGING SOURCE**********'


    miriad.gpcopy(vis=cal_1,out=src_1,mode='copy')

    srcflag(flags1)
    print '.....1st cycle.....'

    srcflag(flags2)
    print '.....2nd cycle.....'

    #srcflag(flags3)
    #print '.....3rd cycle.....'

    #srcflag(flags4)

    #print '.....4th cycle.....'


    print '***********END AUTOFLAGGING SOURCE**********'





***********START AUTOFLAGGING SOURCE**********
.....1st cycle.....
.....2nd cycle.....
***********END AUTOFLAGGING SOURCE**********
***********START AUTOFLAGGING SOURCE**********
.....1st cycle.....
.....2nd cycle.....
***********END AUTOFLAGGING SOURCE**********
***********START AUTOFLAGGING SOURCE**********
.....1st cycle.....
.....2nd cycle.....
***********END AUTOFLAGGING SOURCE**********


In [210]:
# CONTINUUM IMAGE 
for i in xrange(1,4):
    path_or='/Users/maccagni/Sharp_data/PKS/'+obs+'epoch_'+str(i)+'/'
    os.chdir(path_or)
    
    
    
    continuum_image(src_1)

print '***********END CONTINUUM**********'





***********CONTINUUM: Invert**********
***********CONTINUUM: Clean**********
***********CONTINUUM: Restore**********
***********CONTINUUM: Invert**********

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'invert':  Visibilities rejected: 19331830
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'invert':  Visibilities rejected: 18939870
  warnings.warn(msg)



***********CONTINUUM: Clean**********
***********CONTINUUM: Restore**********
***********CONTINUUM: Invert**********
***********CONTINUUM: Clean**********
***********CONTINUUM: Restore**********
***********END CONTINUUM**********


/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'invert':  Visibilities rejected: 19101530
  warnings.warn(msg)


In [211]:
#CUBE

for i in xrange(1,4):
    path_or='/Users/maccagni/Sharp_data/PKS/'+obs+'epoch_'+str(i)+'/'
    os.chdir(path_or)
    
    src_cube='src_cube'
    cal_cube='cal_cube'
    

    cleanup(src_cube)
    cleanup(cal_cube)
    
    shutil.copytree(src_1,src_cube)
    shutil.copytree(cal_1,cal_cube)
    

    make_cube(cal_cube,src_cube)


    print '***********END CUBE**********'
    print i
print "END"

***********cut the band**********
***********continuum subtraction**********
***********VEL: puthd**********
***********invert**********

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 318522
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'invert':  Visibilities rejected: 382
  warnings.warn(msg)



***********END CUBE**********
1
***********cut the band**********
***********continuum subtraction**********
***********VEL: puthd**********
***********invert**********

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 171576
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'invert':  Visibilities rejected: 400
  warnings.warn(msg)



***********END CUBE**********
2
***********cut the band**********
***********continuum subtraction**********
***********VEL: puthd**********
***********invert**********

/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'mfcal':  Correlations flagged or edge-rejected: 99058
 Using post-Aug94 ATCA flux scale for 1934-638
  warnings.warn(msg)
/Library/Python/2.7/site-packages/mirpy-0.3.1-py2.7.egg/mirpy/wrapper.py:83: UserWarning: 'invert':  Visibilities rejected: 232
  warnings.warn(msg)



***********END CUBE**********
3
END


In [212]:
#EXTRACT the spectrum

for i in xrange(1,4):
    path_or='/Users/maccagni/Sharp_data/PKS/'+obs+'epoch_'+str(i)+'/'
    os.chdir(path_or)

    cube='m'
    cubefits='cube.fits'

    miriad.fits(IN=cube,out=cubefits,op='xyout')


    outfile='spectrum.csv'
    infile=cubefits
    v_sys=4274

    px=512
    py=512


    print '***********Extract the Spectrum**********'


    noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)


    infile = outfile
    data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
    #toget the column names
    freq=data['freqHz']
    flux=data['fluxJybeam']
    vel=data['velkms']
    vel_rel=data['vel_relkms']
    noise=data['noise']

    mean_noise=np.mean(noise)


    print '***********Plot the Spectrum**********'


    outfilefig='spectrum.pdf'

    plotta(vel,flux,noise,outfilefig,int(0),int(0),4274,mean_noise)




print 'NORMAL TERMINATION'

***********Extract the Spectrum**********
***********Plot the Spectrum**********
***********Extract the Spectrum**********
***********Plot the Spectrum**********
***********Extract the Spectrum**********
***********Plot the Spectrum**********
NORMAL TERMINATION


In [229]:
#combine all three datasets
path_or='/Users/maccagni/Sharp_data/PKS/'+obs

os.chdir(path_or)

cube1='epoch_1/src_line'
cube2='epoch_2/src_line'
cube3='epoch_3/src_line'

inputfile=cube1+','+cube2+','+cube3

#Invert: make the right cube
#set filenames
cube_im='m_comb'
cube_bm='b_comb' 

cleanup(cube_im)
cleanup(cube_bm)

#shutil.rmtree(cube_im)
#shutil.rmtree(cube_bm)

#set parameters
IMSIZE='1024'
CELL='2'
#CHANNEL='channel,250,1,3,3'
ROBUST='0'
STOKES='ii'
OPTIONS='double'
SLOP='0.8'  
    
#centrato sulla riga
CHANNEL='channel,202,54,1,1'

print '***********invert**********'    
    
miriad.invert(vis=inputfile,map=cube_im,beam=cube_bm,imsize=IMSIZE,
              cell=CELL,robust=ROBUST,stokes=STOKES,options=OPTIONS,slop=SLOP,line=CHANNEL)
    

    
#CLEAN the image
print '***********CUBE: Clean**********'
    

    
#set parameter filenames
clean_cont='cl_cube'
cube_restor='cube_rest'
cube_han='m_comb_han'


cleanup(clean_cont)
cleanup(cube_restor)
cleanup(cube_han)



#set parameters
CUTOFF='0.0004'
NITERS=1000
#MASK='mask(mask_cube)'
miriad.clean(map=cube_im,beam=cube_bm,cutoff=CUTOFF,out=clean_cont,niters=NITERS)
    
#RESTOR the image
print '***********CONTINUUM: Restore**********'
       
#set parameter filenames    
miriad.restor(model=clean_cont,map=cube_im,beam=cube_bm,out=cube_restor)    
    
    
miriad.velsw(IN=cube_im,axis='vopt')    


miriad.hanning(IN=cube_im,out=cube_han,width=3)


print '***********END CUBE**********'


***********invert**********
***********CUBE: Clean**********
***********CONTINUUM: Restore**********
***********END CUBE**********


In [214]:
#EXTRACT the spectrum from the combined dataset

cube='m_comb_han'
cubefits='cube_comb_han.fits'

miriad.fits(IN=cube,out=cubefits,op='xyout')


outfile='spectrum.csv'
infile=cubefits
v_sys=4274

px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)


infile = outfile
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq=data['freqHz']
flux=data['fluxJybeam']
vel=data['velkms']
vel_rel=data['vel_relkms']
noise=data['noise']

mean_noise=np.mean(noise)


print '***********Plot the Spectrum**********'


outfilefig='spectrum.pdf'

plotta(vel,flux,noise,outfilefig,int(0),int(0),4274,mean_noise)




print 'NORMAL TERMINATION'

***********Extract the Spectrum**********
***********Plot the Spectrum**********
NORMAL TERMINATION


In [7]:
#EXTRACT the spectrum from the combined dataset

path_or='/Users/maccagni/Sharp_data/PKS/'

os.chdir(path_or)


print '***********First cube*********'



cubefits1='v1/cube_comb.fits'
cube='v1/m_comb'
miriad.fits(IN=cube,out=cubefits1,op='xyout')
outfile='v1/spectrum_comb1.csv'
infile=cubefits1
v_sys=4274
px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)

infile = 'v1/spectrum_comb1.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq1=data['freqHz']
flux1=data['fluxJybeam']
vel1=data['velkms']
vel_rel1=data['vel_relkms']
noise1=data['noise']

mean_noise1=np.mean(noise1)

print '***********Second cube*********'


cubefits2='v2/cube_comb.fits'
cube='v2/m_comb'
miriad.fits(IN=cube,out=cubefits2,op='xyout')
outfile='v2/spectrum_comb2.csv'
infile=cubefits2
v_sys=4274
px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)
infile = 'v2/spectrum_comb2.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq2=data['freqHz']
flux2=data['fluxJybeam']
vel2=data['velkms']
vel_rel2=data['vel_relkms']
noise2=data['noise']

mean_noise2=np.mean(noise1)


print '***********Third cube*********'


cubefits2='v3/cube_comb.fits'
cube='v3/m_comb'
miriad.fits(IN=cube,out=cubefits2,op='xyout')
outfile='v3/spectrum_comb3.csv'
infile=cubefits2
v_sys=4274
px=512
py=512


print '***********Extract the Spectrum**********'


noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)
infile = 'v3/spectrum_comb3.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq3=data['freqHz']
flux3=data['fluxJybeam']
vel3=data['velkms']
vel_rel3=data['vel_relkms']
noise3=data['noise']

mean_noise2=np.mean(noise1)





print '***********Original cube**********'


cubefits_or='f.fits'

infile=cubefits_or
outfile='spectrum_or.csv'
noise,mean_noise=SpecExtr(infile,outfile,v_sys,px,py)

infile = 'spectrum_or.csv'
data=np.genfromtxt(infile,dtype=float, delimiter=',', names=True)
    
#toget the column names
freq_or=data['freqHz']
flux_or=data['fluxJybeam']
vel_or=data['velkms']
vel_rel_or=data['vel_relkms']
noise_or=data['noise']

mean_noise_or=np.mean(noise_or)



#outfilefig='spectrum_overlay_han.pdf'

#plotta_over(vel1,flux1,vel2,flux2,vel_or,flux_or,resid_x,resid,outfilefig,int(0),int(0),4274,mean_noise1)


#interpolation to subtract the spectra

path_or='/Users/maccagni/Sharp_data/PKS/'


spec_2 = interp1d(vel3, flux3, kind='cubic',bounds_error=False,fill_value=np.nan)

spec_or = interp1d(vel_or, flux_or, kind='cubic',bounds_error=False,fill_value=np.nan)

resid_x= np.linspace(vel2[0],vel2[-1],1e4,endpoint=True)



resid= spec_or(resid_x) - spec_2(resid_x) 


outfilefig=path_or+'spectrum_overlay_1.pdf'

plotta_over(vel1,flux1,vel2,flux2,vel3,flux3,
            vel_or,flux_or,resid_x,resid,outfilefig,int(0),int(0),4274,0.0)

print 'NORMAL TERMINATION'

***********First cube*********
***********Extract the Spectrum**********
***********Second cube*********
***********Extract the Spectrum**********
***********Third cube*********
***********Extract the Spectrum**********
***********Original cube**********
NORMAL TERMINATION


NORMAL TERMINATION
